In [ ]:
# 导入必要的库
import pandas as pd
from datetime import datetime, timedelta

# 简化版本的 parse_smet_file 和辅助函数
def parse_smet_file_content(file_path):
    header = {}
    data = []
    with open(file_path, 'r') as file:
        header_mode = True
        for line in file:
            if line.startswith('[DATA]'):
                header_mode = False
                continue
            if header_mode:
                if '=' in line:
                    parts = line.strip().split('=')
                    header[parts[0].strip()] = parts[1].strip()
            else:
                data.append(line.strip().split())

    if not data:
        return header, pd.DataFrame()

    df = pd.DataFrame(data, columns=header['fields'].split())
    return header, df

def parse_smet_file(file_path, stime, etime, step):
    header, df = parse_smet_file_content(file_path)
    if df.empty:
        return []

    time_column = df.columns[0]
    df[time_column] = pd.to_datetime(df[time_column])

    # 生成请求的时间点
    time_points = []
    current_time = stime
    while current_time <= etime:
        time_points.append(current_time)
        current_time += timedelta(hours=step)

    # 过滤数据并重命名时间列为 'time'
    filtered_df = df[df[time_column].isin(time_points)]
    filtered_df = filtered_df.rename(columns={time_column: 'time'})
    filtered_df['time'] = filtered_df['time'].dt.strftime("%Y-%m-%d %H:%M:%S")
    return filtered_df

# 设置测试参数
file_path = 'path_to_your_file.smet'  # 这里需要替换成实际文件路径
stime = datetime(2025, 4, 1, 0, 0, 0)  # 设置时间范围的开始
etime = datetime(2025, 4, 1, 12, 0, 0)  # 设置时间范围的结束
step = 1  # 设置时间步长为 1 小时

# 读取数据
smet_data = parse_smet_file(file_path, stime, etime, step)

# 查看数据中的雪深部分
snow_depth_columns = ['snow_height(mod)', 'snow_height(meas)']  # 根据实际列名调整
print(smet_data[snow_depth_columns])

In [ ]:
import netCDF4 as nc
import numpy as np

def inspect_wrfout(wrfout_path):
    try:
        ds = nc.Dataset(wrfout_path)
        print("✅ 成功打开文件:", wrfout_path)
        
        # 检查所有变量列表
        print("\n--- 文件包含的变量列表 ---")
        print(list(ds.variables.keys()))
        
        # 检查第一个时间点数据
        print("\n--- 第一个时间点的变量值 ---")
        time_idx = 0  # 第一个时间点
        
        # 1. 检查时间变量
        if 'Times' in ds.variables:
            time_str = ''.join(ds.variables['Times'][time_idx].astype(str))
            print(f"Time (Times): {time_str}")
        else:
            print("❌ 未找到时间变量 'Times'")
        
        # 2. 检查目标变量存在性及数值
        target_vars = [
            'T2', 'SHI3D', 'SNOWH', 'PSFC', 'RAINC', 'FSA', 'FIRA',
            'U', 'V', 'W'
        ]
        
        for var_name in target_vars:
            if var_name in ds.variables:
                var = ds.variables[var_name]
                data = var[time_idx]
                print(f"\n✅ 变量 {var_name} 存在")
                print(f"   维度形状: {data.shape}")
                print(f"   单位: {var.units if hasattr(var, 'units') else '未知'}")
                print(f"   第一个空间点值: {data[0,0,0] if data.ndim >=3 else data[0]}")
            else:
                print(f"\n❌ 变量 {var_name} 不存在")
                
        # 3. 检查风速计算逻辑
        if all(v in ds.variables for v in ['U', 'V', 'W']):
            U = ds.variables['U'][time_idx]
            V = ds.variables['V'][time_idx]
            W = ds.variables['W'][time_idx]
            
            # 注意：WRF 的 U/V 可能位于交错网格
            try:
                speed = np.sqrt(U**2 + V**2 + W**2)
                print("\n✅ 风速计算测试:")
                print(f"   U 形状: {U.shape}, 示例值: {U[0,0,0]}")
                print(f"   V 形状: {V.shape}, 示例值: {V[0,0,0]}")
                print(f"   W 形状: {W.shape}, 示例值: {W[0,0,0]}")
                print(f"   风速示例值: {speed[0,0,0]}")
            except Exception as e:
                print(f"\n❌ 风速计算失败: {str(e)}")
        else:
            print("\n❌ 缺失 U/V/W 变量，无法计算风速")
            
        ds.close()
        
    except Exception as e:
        print("❌ 文件读取失败:", str(e))

# 使用示例（替换为你的 wrfout 文件路径）
wrfout_file = "/WRF/Product_WRF/WRFOUT/wrfout_d01_2025-04-22_00:00:00"
inspect_wrfout(wrfout_file)

❌ 文件读取失败: [Errno 2] No such file or directory: '/home/projects/data/wrf_output/wrfout_d02_2025-04-22_00:00:00'
